# **Detection of Buildings (Test Environment)**

#### **Team Bag – Andrius Šukys, Benas Skripkiūnas, Greta Virpšaitė**

This is the test environment for project Detection of Buildings.

In order to test a model, download a selected model from Google Drive, upload it to path `../models/` of your project and run the REST API interface.

**Do not forget to upload the included file `segmentator.html` to root folder of the project, otherwise the test environment will not work!**

The environment was made by the creators of the project to test it themselves. For this reason, some values are hard-coded and might need to be adjusted individually:

* Mean/Std in Block 2 (in section Transformations).
* Directory/Name of the model and Threshold in Block 6 (in section API).

#### **The source codes and the final report of the project are all available on [GitHub](https://github.com/Andrius-Sukys/DL-DetectionOfBuildings).**

#### **The datasets, predictions and output models due to their big size (GitHub does not allow to save them there) of the project are all available on [Google Drive](https://drive.google.com/drive/folders/1jMvX0dYo60M-s_ZaNAo_Fe_TC1f5Tut7?usp=sharing).**

# Setup of needed Modules and Libraries


In [12]:
from albumentations.pytorch import ToTensorV2
from matplotlib.ticker import MultipleLocator
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import albumentations as alb
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as F

# Transformations

**NOTE: DO NOT FORGET TO UPDATE THE MEAN AND STD!**

In [13]:
mean = [0, 0, 0]
std = [1, 1, 1]
# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]

validation_transform = alb.Compose([alb.Resize(height = 224, width = 224),
                                    alb.Normalize(mean = mean, std = std, max_pixel_value = 255.0),
                                    ToTensorV2()])

# Model based on UNet Architecture

In [14]:
# Generally used to deepen the neural network for better performance
class DoubleConvolution(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    # Double convolution as a sequence of operations applied to data
    self.double_conv = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size = 3, padding = 1), # 2D convolution
                                     nn.BatchNorm2d(out_channels), # Batch normalization
                                     nn.ReLU(inplace = True), # The original input is modified instead of creating a new tensor
                                     nn.Conv2d(out_channels, out_channels, kernel_size = 3, padding = 1), # 2D convolution
                                     nn.BatchNorm2d(out_channels), # Batch normalization
                                     nn.ReLU(inplace = True)) # The original input is modified instead of creating a new tensor

  def forward(self, x):
    return self.double_conv(x)

# Used to reduce the dimensions of input feature maps and increase receptive field
class DownSample(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    # Performs max pooling and double convolution on reduced feature maps while increasing channels
    self.maxpool_conv = nn.Sequential(nn.MaxPool2d(2), DoubleConvolution(in_channels, out_channels))

  def forward(self, x):
    return self.maxpool_conv(x)

# Used to increase the dimensions of the downsampled feature maps
class UpSample(nn.Module):
  def __init__(self, in_channels, out_channels):
    super().__init__()

    # Used to increase the spatial dimensions of feature maps while decreasing the number of channels
    # The feature map grows twice each time because of kernel_size and stride but the channels get reduced
    self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size = 2, stride = 2)
    self.conv = DoubleConvolution(in_channels, out_channels)

  def forward(self, x1, x2):
    x1 = self.up(x1)

    # x2 is cropped to match the dimensions of x1
    # shape[1] is the number of channels, shape[2] is for height, shape[3] is for width
    x2 = F.center_crop(x2, [x1.shape[2], x1.shape[3]])

    # Combines both feature maps to preserve both deep and shallow level details, concatenated horizontally
    x1 = torch.cat([x2, x1], dim = 1)

    return self.conv(x1)

class UNet(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(UNet, self).__init__()
    self.inc = (DoubleConvolution(in_channels, 64))
    self.down1 = (DownSample(64, 128))
    self.down2 = (DownSample(128, 256))
    self.down3 = (DownSample(256, 512))
    self.down4 = (DownSample(512, 1024))
    self.up1 = (UpSample(1024, 512))
    self.up2 = (UpSample(512, 256))
    self.up3 = (UpSample(256, 128))
    self.up4 = (UpSample(128, 64))
    # Used to decrease the number of channels so the kernel_size is 1
    self.outc = nn.Conv2d(64, out_channels, kernel_size = 1)

  def forward(self, x):
    x1 = self.inc(x)
    x2 = self.down1(x1)
    x3 = self.down2(x2)
    x4 = self.down3(x3)
    x5 = self.down4(x4)
    x = self.up1(x5, x4)
    x = self.up2(x, x3)
    x = self.up3(x, x2)
    x = self.up4(x, x1)
    logits = self.outc(x)
    return logits

# Initialization of Device

Recommended to use GPU *(cuda:0)* for optimal performance.


In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Used device type is', device)

Used device type is cuda:0


## API ##

API is used for testing purposes. A user may upload an image and get the mask of buildings back.

**File segmentator.html must be added to the root folder for the API interface to work!**

**Likewise, a model must be provided!**

Use the link as the API interface.

In [16]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://jsuzdk2cf2l-496ff2e9c6d22116-5000-colab.googleusercontent.com/


Selecting the threshold and loading the uploaded model.

**NOTE: DO NOT FORGET TO UPDATE THE MODEL_PATH!**

**NOTE: DO NOT FORGET TO UPDATE THE THRESHOLD!**

In [17]:
model = UNet(in_channels = 3, out_channels = 1).to(device)
model_path = f'../models/UNet_Threshold_0.4.pth'

threshold = 0.4

model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Using Flask to handle user requests.

In [18]:
from flask import Flask, render_template, request
import torchvision.transforms as transforms
import io
import base64

app = Flask(__name__, template_folder='../')

@app.route('/', methods=['GET', 'POST'])
def index():
    original_image_base64 = None
    mask_base64 = None

    if request.method == 'POST':

        file = request.files['file']
        original_image = np.array(Image.open(file).convert("RGB"))

        transformed_image = validation_transform(image = original_image)['image']

        predicted_mask = model(torch.unsqueeze(transformed_image, 0).to(device))
        predicted_mask = torch.sigmoid(predicted_mask)

        # Change the threshold to a desired one!
        predicted_mask = (predicted_mask > threshold).float().cpu().squeeze().numpy()

        mask_pil = transforms.ToPILImage()(predicted_mask)

        original_image_buffered = io.BytesIO()
        mask_buffered = io.BytesIO()

        Image.fromarray(original_image).save(original_image_buffered, format = "PNG")
        mask_pil.save(mask_buffered, format = "PNG")

        original_image_base64 = base64.b64encode(original_image_buffered.getvalue()).decode('utf-8')
        mask_base64 = base64.b64encode(mask_buffered.getvalue()).decode('utf-8')

    # Don't forget to upload segmentator.html to main folder!
    return render_template('segmentator.html', original_image = original_image_base64, mask = mask_base64)

if __name__ == "__main__":
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:43:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:43:31] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:43:38] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:43:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:43:49] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:43:49] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:46:13] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:46:15] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:46:50] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/May/2024 13:46:52] "GET /favicon.ico HTTP/1.1" 404